In [ ]:
Sys.time()

# Index IUC calls
This is all IUC calls with a dental disposition, filtered on possible outbound portion of postcodes found in the dental dataset since the dental data only covers BD and LS postoces, I think. Date range is 1st October 2022 and 31st October, 2023.

**NOTE** Jaison has advised that we only have dental data for BD and LS postcode areas. Will filter iuc data on unqiue PARTIAL_POSTCODEs in the dental datasetsince the IUC dataset already only contains cases within the date range and for the previously provided final dispostion codes consistent with urgent dental care.



In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8) 
options(tibble.width = Inf, width = 300)

In [ ]:
library(DBI)
library(dbplyr)
library(bigrquery)
suppressPackageStartupMessages(library(tidyverse))
library(lubridate)
library(readxl)
library(knitr)
library(IRdisplay)
library(htmltools)

In [ ]:
con <- DBI::dbConnect(
    bigquery(), 
    project = "yhcr-prd-bradfor-bia-core", 
    bigint = "character", 
    page_size = 5000,
    dataset = 'CB_2617'
)


In [ ]:
dental_tbl <- tbl(con, "tbl_Dental_Data_NHBSA")

In [ ]:
dental_df <- dental_tbl %>% filter(between(tbl_Dental_Data_NHBSA_start_date, '2022-10-01', '2023-10-31')) %>% collect()

In [ ]:
unique_dental_pc <- paste0("'",dental_df %>% count(PARTIAL_POSTCODE) %>% pull(PARTIAL_POSTCODE), collapse = "',")

# Fetch IUC cases where home postcode is either BD or LS

In [ ]:
unique_dental_pc

In [ ]:
iuc_sql <- glue::glue("
WITH base_data AS (
  SELECT *
  FROM `tbl_YAS_IUC`
  WHERE person_id IS NOT NULL
    AND patient_current_postcode IN ({unique_dental_pc}')
),

ordered_calls AS (
  SELECT
    *,
    LAG(tbl_YAS_IUC_start_date) OVER (
      PARTITION BY person_id
      ORDER BY tbl_YAS_IUC_start_date
    ) AS previous_call
  FROM base_data
),

session_marks AS (
  SELECT
    *,
    CASE
      WHEN previous_call IS NULL THEN 1
      WHEN DATE_DIFF(tbl_YAS_IUC_start_date, previous_call, DAY) > 7 THEN 1
      ELSE 0
    END AS new_session
  FROM ordered_calls
),

sessions AS (
  SELECT
    *,
    SUM(new_session) OVER (
      PARTITION BY person_id
      ORDER BY tbl_YAS_IUC_start_date
    ) AS session_id
  FROM session_marks
),

-- Identify the index caseref (first call per session)
index_calls AS (
  SELECT
    person_id,
    session_id,
    tbl_YAS_IUC_start_date AS index_call_date,
    caseref AS index_call_id
  FROM (
    SELECT
      person_id,
      session_id,
      tbl_YAS_IUC_start_date,
      caseref,
      ROW_NUMBER() OVER (
        PARTITION BY person_id, session_id
        ORDER BY tbl_YAS_IUC_start_date
      ) AS rn
    FROM sessions
  )
  WHERE rn = 1
)

-- Final output
SELECT
  s.person_id,
  s.tbl_YAS_IUC_start_date,
  s.tbl_YAS_IUC_end_date,
  s.caseref,
  s.gp_surgery_id,
  s.patient_home_postcode,
  s.first_pathway_selected,
  s.final_dx_description,
  s.final_symptom_group,
  s.final_symptom_discriminator,
  s.referral_service,
  s.referral_service_type,
  s.direct_booking_service,
  s.rejected_service,
  s.rejected_service_reason,
  ic.index_call_date,
  ic.index_call_id,
  CASE WHEN s.caseref = ic.index_call_id THEN 1 ELSE 0 END AS is_index_call
FROM sessions s
JOIN index_calls ic
  ON s.person_id = ic.person_id
  AND s.session_id = ic.session_id
ORDER BY s.person_id, s.tbl_YAS_IUC_start_date;
")

In [ ]:
iuc_df <- dbGetQuery(con, iuc_sql)

In [ ]:
html_output <- paste0(
  '<div style="overflow-x: auto; white-space: nowrap;">',
  kable(iuc_df %>% head(), format = "html"),
  '</div>'
)

display_html(html_output)

## Total index calls

In [ ]:
iuc_df %>% count(is_index_call)

## IUC calls per month

In [ ]:
iuc_df %>%
    #filter(is_index_call == 1) %>%
    mutate(month = floor_date(tbl_YAS_IUC_start_date, "month")) %>%
      count(month, is_index_call) %>%
      arrange(month) %>%
        ggplot(aes(x = month, y = n, fill = is_index_call)) +
        geom_col()

## Associate dental cases with index IUC calls

In [ ]:
index_iuc <- iuc_df %>%
  filter(is_index_call == 1) %>%
  select(
      person_id, 
      index_call_date = tbl_YAS_IUC_start_date, 
      index_caseref = caseref,
      index_referral_service = referral_service,
      index_disposition = final_dx_description
    )

In [ ]:
index_iuc %>% glimpse()
index_iuc %>% saveRDS('../data/index_iuc_df.rds')

## Write Index IUC calls to datbase to inner join on other tables

In [ ]:
#DBI::dbWriteTable(con, "index_iuc", index_iuc, overwrite = TRUE)

In [ ]:
dental_tagged <- dental_df %>%
  inner_join(index_iuc, by = "person_id", relationship = "many-to-many") %>%
  filter(tbl_Dental_Data_NHBSA_start_date >= index_call_date,
         tbl_Dental_Data_NHBSA_start_date <= index_call_date + days(7))

In [ ]:
dental_tagged %>% count()

In [ ]:
colnames(dental_tagged)

In [ ]:
visit_in_time_df <- dental_tagged %>%
  mutate(
    day_diff = as.integer(as_date(tbl_Dental_Data_NHBSA_start_date) - as_date(index_call_date))
  ) %>%
  group_by(index_call_date, person_id) %>%
  summarise(
    n = n(),
    dental_within_24hr = sum(day_diff >= 0 & day_diff <= 1),
    dental_within_7d = sum(day_diff >= 0 & day_diff <= 7)
  ) %>%
  ungroup()

In [ ]:
visit_in_time_df %>% glimpse()

In [ ]:
visit_in_time_df %>%
    transmute(
        month = floor_date(index_call_date, unit = 'month'),
        dental_within_24hr = if_else(dental_within_24hr >= 1, 'yes', 'no'),
        dental_within_7d = if_else(dental_within_7d >= 1, 'yes', 'no')
    ) %>%
    pivot_longer(
    cols = c(dental_within_24hr, dental_within_7d),
    names_to = "window",
    values_to = "in_time"
  ) %>%
    ggplot(aes(x = month, fill = in_time)) +
    geom_bar(position="stack")+
     facet_wrap(~window)

# All IUC dental vs actual dental contact

In [ ]:
index_iuc %>% glimpse()

In [ ]:
dental_tagged %>% glimpse()

In [ ]:
all_iuc_dental_df <- index_iuc %>%
    left_join(dental_tagged %>% mutate(dental = 1) %>% select(dental, index_caseref, EXEMPTION_REMISSION_FLAG, TREATMENT_CHARGE_BAND, NICE_RECALL_GROUP), by="index_caseref") %>%
    arrange(index_caseref, index_call_date) %>%
    group_by(index_caseref, index_call_date) %>%
    slice(1) %>% # Only return first treatment for this
    ungroup()

In [ ]:
all_iuc_dental_df %>% glimpse()

In [ ]:
all_iuc_dental_df %>% mutate(dental = if_else(is.na(dental), 0, dental)) %>% count(dental, index_disposition, sort = T) %>% head(10)
# Note that dental denotes whether there was a dental contact

In [ ]:
iuc_dental_referral_service_df <- all_iuc_dental_df %>% 
    mutate(
        dental_contact = as.factor(if_else(is.na(dental), 'no', 'yes')),
        referral_group = case_when(
            grepl('Dental CAS', index_referral_service) ~ 'Dental CAS',
            grepl('ED:', index_referral_service) ~ 'ED',
            grepl('GP OOH', index_referral_service) ~ 'GP OOH',
            grepl('GP', index_referral_service) ~ 'GP in-hours',
            grepl('Dental Emergency', index_referral_service) ~ 'Dental Emergency',
            grepl('Dentist|Dental', index_referral_service) ~ 'Dentist',
            grepl('YAS NHS111', index_referral_service) ~ '111 ED verification',
            grepl('UTC|Urgent Care', index_referral_service) ~ 'UTC',
            .default = 'Other'
        )
    ) %>% 
    count(dental_contact, referral_group, sort = T)

In [ ]:
iuc_dental_referral_service_df %>%
    pivot_wider(names_from = 'dental_contact', values_from = 'n')

In [ ]:
all_iuc_dental_summary_df <- all_iuc_dental_df %>%
    transmute(
        dental_contact = as.factor(if_else(is.na(dental), 'no', 'yes')),
        EXEMPTION_REMISSION_FLAG, 
        TREATMENT_CHARGE_BAND,NICE_RECALL_GROUP
    )

In [ ]:
all_iuc_dental_summary_df %>%
    count(dental_contact, EXEMPTION_REMISSION_FLAG, sort = T) %>% head(10)

In [ ]:
all_iuc_dental_summary_df %>% count(dental_contact, NICE_RECALL_GROUP, sort = T) %>% head(10)

In [ ]:
all_iuc_dental_summary_df %>% count(dental_contact, TREATMENT_CHARGE_BAND, sort = T) %>% head(10)